

We are using YOLO for detection. So we adjust the dataset to its format.


Imports necessary libraries: os (file operations), pandas (data handling), and cv2 (image processing).

Reads the CSV file containing annotations for license plate detection.

Specifies folders where images and labels are stored.

Ensures the label folder exists, creating it if necessary.

Iterates through the CSV file and reads image annotations.

Converts bounding box coordinates to YOLO format:
(x_center, y_center): Center of the bounding box (normalized).
(bbox_width, bbox_height): Width and height of the bounding box (normalized).

Saves the converted labels in a .txt file, one per image.

Output: "CSV converted to YOLO format!" VOILA! successful conversion.


In [1]:
import os
import pandas as pd
import cv2

# Load CSV file
csv_path = "Licplatesdetection_train.csv"  # Update with actual path
df = pd.read_csv(csv_path)

# Define input/output folders
image_folder = "data_detection/images"
label_folder = "data_detection/labels"

os.makedirs(label_folder, exist_ok=True)

# Convert CSV to YOLO format
for index, row in df.iterrows():
    img_path = os.path.join(image_folder, row["img_id"])
    img = cv2.imread(img_path)
    h, w, _ = img.shape  # Get image dimensions

    # Convert bounding box to YOLO format
    xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]
    x_center = (xmin + xmax) / (2 * w)
    y_center = (ymin + ymax) / (2 * h)
    bbox_width = (xmax - xmin) / w
    bbox_height = (ymax - ymin) / h

    # YOLO format: class_id x_center y_center width height
    label_path = os.path.join(label_folder, row["img_id"].replace(".jpg", ".txt"))
    with open(label_path, "w") as f:
        f.write(f"0 {x_center} {y_center} {bbox_width} {bbox_height}\n")  # Class 0 for license plates

print("CSV converted to YOLO format!")


CSV converted to YOLO format!


In [ ]:
import os
import shutil
import random

# Define paths
train_img_folder = "data_detection/train/images"
train_lbl_folder = "data_detection/train/labels"
val_img_folder = "dataset/val/images"
val_lbl_folder = "dataset/val/labels"

# Create validation folders
os.makedirs(val_img_folder, exist_ok=True)
os.makedirs(val_lbl_folder, exist_ok=True)

# Get all images
all_images = os.listdir(train_img_folder)
random.shuffle(all_images)

# Move 20% of images to validation
val_size = int(0.2 * len(all_images))
val_images = all_images[:val_size]

for img_name in val_images:
    # Move image
    shutil.move(os.path.join(train_img_folder, img_name), os.path.join(val_img_folder, img_name))

    # Move corresponding label file
    label_name = img_name.replace(".jpg", ".txt")  # Adjust extension if needed
    shutil.move(os.path.join(train_lbl_folder, label_name), os.path.join(val_lbl_folder, label_name))

print("Validation set created successfully!")


Validation set created successfully!


In [6]:
from ultralytics import settings
settings.reset()  # Clears cached dataset paths


In [7]:
import os

train_path = "D:/Projects/DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001/data_detection/train/images"
val_path = "D:/Projects/DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001/data_detection/val/images"

print("Train path exists:", os.path.exists(train_path))
print("Validation path exists:", os.path.exists(val_path))


Train path exists: True
Validation path exists: True


In [8]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Load YOLO model
model.train(data="data-l.yaml", epochs=50, imgsz=640, batch=16)


New https://pypi.org/project/ultralytics/8.3.92 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.15  Python-3.12.4 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data-l.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning D:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\train\labels... 720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<00:00, 1369.84it/s]

train: New cache created: D:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\train\labels.cache



val: Scanning D:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\val\labels... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<00:00, 1796.62it/s]

val: New cache created: D:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\val\labels.cache


...

                   all        180        180      0.994          1      0.995      0.805



50 epochs completed in 2.993 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 5.6MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 5.6MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.15  Python-3.12.4 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 186 layers, 2,684,563 parameters, 0 gradients, 6.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.31s/it]


                   all        180        180      0.994          1      0.995      0.804
Speed: 0.7ms preprocess, 35.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019BE69ADAC0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [9]:
model = YOLO("runs/detect/train4/weights/best.pt")
results = model.predict(source="data_detection/test", save=True)



image 1/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\test\1000.jpg: 448x640 1 license_plate, 141.9ms
image 2/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\test\1001.jpg: 640x384 1 license_plate, 62.4ms
image 3/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\test\1002.jpg: 640x480 1 license_plate, 66.9ms
image 4/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\test\1003.jpg: 384x640 1 license_plate, 50.3ms
image 5/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\test\1004.jpg: 640x384 1 license_plate, 40.8ms
image 6/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\test\1005.jpg: 640x384 1 license_plate, 31.4ms
image 7/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-001\data_detection\test\1006.jpg: 640x480 1 license_plate, 35.3ms
image 8/210 d:\Projects\DATA SCIENTIST_ASSIGNMENT-20250318T115327Z-

In [6]:
!pip install torch torchvision torchaudio numpy opencv-python pillow pytorch-lightning Levenshtein


In [5]:
import os
import pandas as pd
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Define character mapping (A-Z, 0-9)
characters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
char_to_index = {char: idx + 1 for idx, char in enumerate(characters)}  # 1-based index
char_to_index["PAD"] = 0  # Padding character

# Define max sequence length (maximum license plate length)
MAX_LEN = 10

# Define image transformations
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((32, 128)),  # Resize images to standard size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class LicensePlateDataset(Dataset):
    def __init__(self, csv_file, image_folder):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Load image
        img_name = os.path.join(self.image_folder, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        image = transform(image)

        # Encode text label
        text = self.data.iloc[idx, 1].upper()
        label = [char_to_index[char] for char in text if char in char_to_index]

        # Pad label to MAX_LEN
        label += [char_to_index["PAD"]] * (MAX_LEN - len(label))

        return image, torch.tensor(label, dtype=torch.long)

# Define dataset paths
train_csv = "Licplatesrecognition_train.csv"
train_img_folder = "license_plates_recognition_train"

# Create dataset
train_dataset = LicensePlateDataset(train_csv, train_img_folder)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [6]:
import torch.nn as nn

class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()

        # Feature extractor (CNN)
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2,1)),  # Maintain width for sequence processing
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2,1)),
        )

        # LSTM for sequence modeling
        self.lstm = nn.LSTM(512, 256, bidirectional=True, num_layers=2, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv_layers(x)  # CNN features -> (batch, channels, height, width)
        
        x = x.view(x.size(0), x.size(1), -1)  # Flatten height
  # Ensure height is removed, now (batch, channels, width)
        
        x = x.permute(0, 2, 1)  # Correct order for LSTM (batch, sequence, features)

        x, _ = self.lstm(x)  # LSTM for sequence learning
        
        x = self.fc(x)  # Fully connected output
        return x


In [7]:
import torch.optim as optim

# Define model, optimizer, and loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CRNN(num_classes=len(char_to_index)).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CTCLoss(blank=0)  # Use padding as blank character

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Compute loss
        input_lengths = torch.full((outputs.size(0),), outputs.size(1), dtype=torch.long).to(device)
        target_lengths = torch.sum(labels != 0, dim=1).to(device)  # Non-zero lengths

        loss = criterion(outputs.permute(1, 0, 2), labels, input_lengths, target_lengths)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Save the trained model
torch.save(model.state_dict(), "crnn_best.pth")
print("Training complete! Model saved.")


Epoch [1/10], Loss: 2.4026
Epoch [2/10], Loss: 3.0824
Epoch [3/10], Loss: 2.5619
Epoch [4/10], Loss: 2.6114
Epoch [5/10], Loss: 2.5970
Epoch [6/10], Loss: 2.5884
Epoch [7/10], Loss: 2.5755
Epoch [8/10], Loss: 2.5662
Epoch [9/10], Loss: 2.5486
Epoch [10/10], Loss: 2.5266
Training complete! Model saved.


In [8]:

from PIL import Image
from torchvision import transforms
# Load trained CRNN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
crnn_model = CRNN(num_classes=len(char_to_index)).to(device)
crnn_model.load_state_dict(torch.load("crnn_best.pth"))  # Load trained weights
crnn_model.eval()

# Define transformation for OCR
ocr_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((32, 128)),  # Ensure same input size as training
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


# Process each cropped plate
output_folder = "dataset"
cropped_plates = os.listdir(output_folder)

for plate_name in cropped_plates:
    plate_path = os.path.join(output_folder, plate_name)
    
    # Load and preprocess image
    plate_img = Image.open(plate_path).convert('L')
    plate_img = ocr_transform(plate_img).unsqueeze(0).to(device)

    # Predict text
    with torch.no_grad():
        output = crnn_model(plate_img)
        predicted_indices = torch.argmax(output, dim=2).cpu().numpy()[0]
        predicted_text = "".join([characters[p-1] for p in predicted_indices if 0 < p <= len(characters)])

    print(f"Image: {plate_name}, OCR Prediction: {predicted_text}")  # Debugging output



Image: 1000.jpg, OCR Prediction: 
Image: 1001.jpg, OCR Prediction: 
Image: 1002.jpg, OCR Prediction: 
Image: 1003.jpg, OCR Prediction: 
Image: 1004.jpg, OCR Prediction: 
Image: 1005.jpg, OCR Prediction: 
Image: 1006.jpg, OCR Prediction: 
Image: 1007.jpg, OCR Prediction: 
Image: 1008.jpg, OCR Prediction: 
Image: 1009.jpg, OCR Prediction: 
Image: 1010.jpg, OCR Prediction: 
Image: 1011.jpg, OCR Prediction: 
Image: 1012.jpg, OCR Prediction: 
Image: 1013.jpg, OCR Prediction: 
Image: 1014.jpg, OCR Prediction: 
Image: 1015.jpg, OCR Prediction: 
Image: 1016.jpg, OCR Prediction: 
Image: 1017.jpg, OCR Prediction: 
Image: 1018.jpg, OCR Prediction: 
Image: 1019.jpg, OCR Prediction: 
Image: 1020.jpg, OCR Prediction: 
Image: 1021.jpg, OCR Prediction: 
Image: 1022.jpg, OCR Prediction: 
Image: 1023.jpg, OCR Prediction: 
Image: 1024.jpg, OCR Prediction: 
Image: 1025.jpg, OCR Prediction: 
Image: 1026.jpg, OCR Prediction: 
Image: 1027.jpg, OCR Prediction: 
Image: 1028.jpg, OCR Prediction: 
Image: 1029.jp

In [9]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms

# Define character set (A-Z, 0-9)
characters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
char_to_index = {char: idx for idx, char in enumerate(characters)}

# Load trained CRNN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
crnn_model = CRNN(num_classes=len(char_to_index) + 1).to(device)  # +1 for PAD character
crnn_model.load_state_dict(torch.load("crnn_best.pth"))
crnn_model.eval()

ocr_transform = transforms.Compose([
    transforms.Grayscale(),  # Ensure grayscale
    transforms.Resize((32, 128)),  # Resize to training dimensions
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Must match training
])

# Test preprocessing on a sample image
from PIL import Image
plate_img = Image.open("dataset/1000.jpg").convert('L')
plate_img = ocr_transform(plate_img)

print("Preprocessed Image Shape:", plate_img.shape)  # Expected: (1, 32, 128)




# Define test images directory
cropped_plate_folder = "dataset"
output_csv = "ocr_output.csv"

# Prepare results storage
results = []

# Process each cropped plate image
for plate_name in os.listdir(cropped_plate_folder):
    plate_path = os.path.join(cropped_plate_folder, plate_name)
    
    # Load and preprocess image
    plate_img = Image.open(plate_path).convert('L')
    plate_img = ocr_transform(plate_img).unsqueeze(0).to(device)

    # Predict text using CRNN
    with torch.no_grad():
        output = crnn_model(plate_img)
        predicted_indices = torch.argmax(output, dim=2).cpu().numpy()[0]
        predicted_text = "".join([characters[p] for p in predicted_indices if p < len(characters)])

    # Convert to one-hot encoding
    one_hot = [0] * 10  # Initialize 10-character empty list
    for i, char in enumerate(predicted_text[:10]):  # Limit to 10 characters
        if char in char_to_index:
            one_hot[i] = char_to_index[char]  # Assign the index

    # Append results
    results.append([plate_name] + one_hot)

# Convert to DataFrame and Save as CSV
df = pd.DataFrame(results, columns=["id"] + [str(i) for i in range(10)])
df.to_csv(output_csv, index=False)

print(f"OCR results saved to {output_csv} ✅")


Preprocessed Image Shape: torch.Size([1, 32, 128])
OCR results saved to ocr_output.csv ✅


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
crnn_model = CRNN(num_classes=37).to(device)

optimizer = torch.optim.Adam(crnn_model.parameters(), lr=0.001)
criterion = nn.CTCLoss(blank=0)  # Use 0 for blank character

num_epochs = 10
for epoch in range(num_epochs):
    crnn_model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = crnn_model(images)

        input_lengths = torch.full((outputs.size(0),), outputs.size(1), dtype=torch.long).to(device)
        target_lengths = torch.sum(labels != 0, dim=1).to(device)

        loss = criterion(outputs.permute(1, 0, 2), labels, input_lengths, target_lengths)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Save model
torch.save(crnn_model.state_dict(), "crnn_best.pth")
print("✅ Retraining complete!")


Epoch [1/10], Loss: 2.4597
Epoch [2/10], Loss: 3.1923
Epoch [3/10], Loss: 2.5816
Epoch [4/10], Loss: 2.6029
Epoch [5/10], Loss: 2.6147
Epoch [6/10], Loss: 2.6078
Epoch [7/10], Loss: 2.6001
Epoch [8/10], Loss: 2.5929
Epoch [9/10], Loss: 2.5869
Epoch [10/10], Loss: 2.5571
✅ Retraining complete!


In [14]:
plate_img = Image.open("dataset/911.jpg").convert('L')
plate_img = ocr_transform(plate_img).unsqueeze(0).to(device)

with torch.no_grad():
    output = crnn_model(plate_img)
    predicted_indices = torch.argmax(output, dim=2).cpu().numpy()[0]
    predicted_text = "".join([characters[p-1] for p in predicted_indices if 0 < p <= len(characters)])

print("OCR Output for known_good_plate.jpg:", predicted_text)


OCR Output for known_good_plate.jpg: 


In [1]:
import os
import easyocr
import pandas as pd

# Define paths
test_folder = "dataset"
onehot_csv = "ocr_onehot.csv"
plain_csv = "ocr_plain.csv"

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# List all image files in the test folder
image_files = [f for f in os.listdir(test_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Store results
ocr_onehot_results = []
ocr_plain_results = []

# Header row for one-hot encoded CSV
onehot_columns = ["id"] + [str(i) for i in range(10)]

# Process each image
for img_name in image_files:
    img_path = os.path.join(test_folder, img_name)

    # Perform OCR using EasyOCR
    result = reader.readtext(img_path)

    # Extract detected text
    detected_text = "".join([d[1] for d in result]) if result else ""

    # Store non-one-hot encoded result
    ocr_plain_results.append([img_name, detected_text])

    # Initialize one-hot encoding for digits 0-9
    digit_counts = {str(i): 0 for i in range(10)}

    # Update one-hot encoding based on detected digits
    found_digit = False
    for char in detected_text:
        if char.isdigit():
            digit_counts[char] = 1
            found_digit = True

    # Create row data
    row_data = [img_name] + list(digit_counts.values()) if found_digit else [img_name] + [""] * 10
    ocr_onehot_results.append(row_data)

# Save one-hot encoded results to CSV
df_onehot = pd.DataFrame(ocr_onehot_results, columns=onehot_columns)
df_onehot.to_csv(onehot_csv, index=False)

# Save plain text results to CSV
df_plain = pd.DataFrame(ocr_plain_results, columns=["id", "Detected_Text"])
df_plain.to_csv(plain_csv, index=False)

print(f"One-hot encoded OCR results saved to {onehot_csv}")
print(f"Plain text OCR results saved to {plain_csv}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


One-hot encoded OCR results saved to ocr_onehot.csv
Plain text OCR results saved to ocr_plain.csv
